In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import lines
import matplotlib.collections as collections
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_score
# mape is from scikit V0.24, unstable, taken from nightly build
from sklearn.metrics import r2_score, mean_squared_error as mse, mean_absolute_percentage_error as mape
from scipy.stats import f, pearsonr, linregress
from scipy.signal import argrelmax

from helpers import cal_val_split, get_line_ends
from stats import rmspe

In [2]:
pd.__version__

'1.1.2'

In [2]:
plt.tick_params(labelsize=16)

In [3]:
COLORS = {
    #57: '#ff98a6',
    #64: '#3dae46',
    69: '#4270dd',
    74: '#d44958',
    76: '#3dae46',
    81: '#a22bdd',
    87: '#cc7929',
    96: '#f0e521',
    102: '#796f00'
}
COLS = 2
SIZE = 5

In [4]:
data = pd.read_csv("20201025_Dataset_plots_n216.csv")
for col in ['Ca', 'Cb', 'TChl']:
    data[col] = data[col]*1000

In [5]:
def findOptimalLoadings(X, y, maxLoadings):
    scores = np.zeros((maxLoadings, 2))
    for i in range(len(scores)):
        ncomp = i+1
        pls = PLSRegression(n_components=ncomp, scale=False) # range starts from 0
        cv=10
        scores[i] = ncomp, cross_val_score(pls, X, y, cv=cv, scoring='neg_mean_squared_error').sum()
    # argrelmax instead of argelmin - looking for min MSE but scoring returns -MSE. So search for max -MSE
    # argrelmax can return many minima. get the first local minima [0][0]
    try:
        min_ind = argrelmax(scores[:, 1])[0][0]
    except IndexError: # no max
        min_ind = 0
    return scores[min_ind, 0].astype(int)

In [22]:
_filter = True
flagData = data[(data.LeafType == 'FlagLeaf')].copy()
if _filter:
    modifier = "filter"
    flagData = flagData.drop(columns=map(str, range(329, 400)))
else:
    modifier = 'full'
    flagData = flagData.drop(columns=map(str, range(329, 330)))
wls_columns = flagData.columns[11:]
flagData['Ratio'] = flagData.Ca/flagData.Cb
param_data = flagData[flagData.columns.difference(wls_columns)]
wls_data = flagData[wls_columns]
cal, val = cal_val_split(param_data)

In [23]:
def runModel(cal, val, predict, wls, maxLoadings=10):
    '''
    select the important variables for the model
    cal - calibration set (boolean series)
    val - validation set (boolean series)
    predit - column containing variable to predict
    wls - dataframe containing X variables of model (wavelengths)
    loadings - max ammount of loadings in the PLS model
    '''
    name = predict.name
    # R2, RMSE, is validation, is selection
    dataStore = []
    Xcal = wls[cal]
    Xval = wls[val]
    ycal = predict[cal]
    yval = predict[val]
    
    loadings = findOptimalLoadings(Xcal, ycal, maxLoadings)
    pls = PLSRegression(n_components = loadings, scale=False)
    pls.fit(Xcal, ycal)
    calPredict = pls.predict(Xcal)    
    cal_r2 = pls.score(Xcal, ycal)
    cal_rmse =  mse(ycal, calPredict, squared=False)
    cal_rmspe = rmspe(ycal, cal_rmse)
    
    valPredict = pls.predict(Xval)
    val_r2 = pls.score(Xval, yval)
    val_rmse =  mse(yval, valPredict, squared=False)
    val_rmspe = rmspe(yval, val_rmse)

    dataStore.append([name, cal_r2, cal_rmse, cal_rmspe, val_r2, val_rmse, val_rmspe])
    estimates = pd.concat([pd.DataFrame({'obs': obs,
                               'pred': pred,
                               'Type': np.full(obs.shape, name),
                               'cal': np.full(obs.shape, cal)})
                 for obs, pred, cal in ((ycal ,calPredict.flatten(), 1),(yval ,valPredict.flatten(), 0))])
    estimates.index.name = 'org_index'
    estimates = estimates.reset_index()
    coefs = pd.DataFrame(columns=wls.columns, data=pls.coef_.reshape(1, -1))
    coefs['Type'] = name

    collection = pd.DataFrame(data=dataStore, columns=['Type', 'R2C', 'RMSEC', 'RMSPEC', 'R2V', 'RMSEV', 'RMSPEV'])
    return collection, estimates, coefs

In [24]:
def populateModels(cal, val, predicteds, excluders, wls):
    '''
    run model based on the same cal-val split for all the predicteds values
    cal - calibration boolean series selector
    val - validation boolean series selector
    predicteds - dataframe in which each column is to be predicteds
    excluders = dataframe in which each column can be used to exclude samples (union with cal & val)
    wls - X variables (wavelenghts).
    ====
    predictes, excluders & wls come from the same original dataframe, flagData
    CAUTION! This function has HARD-CODED exclusions that change the cal/val split or remove points
    '''
    estimate_data = []
    stats = []
    coef_data = []
    for col, predict in predicteds.iteritems():
        include = ~(excluders.NotAnalizedFor == col) # if True should stay
        ncal = cal & include
        nval = val & include
        if col == 'Cb':
            switcher = (excluders.Plots == 517) & (excluders.DAT == 81) 
            ncal = ncal & ~switcher # turn off switcher
            nval = nval | switcher # turn on switcher

        data, estimates, coefs =  runModel(ncal, nval, predict, wls)
        coef_data.append(coefs)
        stats.append(data)
        estimate_data.append(estimates)
    
    all_stats = pd.concat(stats, ignore_index=True)
    all_coefs = pd.concat(coef_data, ignore_index=True)
    all_estimates = pd.concat(estimate_data, ignore_index=True)
    return all_stats, all_estimates, all_coefs


In [25]:
predicteds = param_data[['Ca', 'Cb', 'TChl']]
excluders = param_data[['Plots', 'DAT', 'NotAnalizedFor']]
stats, estimates, coefs = populateModels(cal, val, predicteds, excluders, wls_data)

In [26]:
dated_estimates = estimates.join(param_data.DAT, on='org_index').drop(columns = ['org_index'])

In [43]:
SAVE_CSV = True
if SAVE_CSV:
    coefs.to_csv(f"./coef tables/{modifier}_coefs.csv", ",", index=False)
    stats.to_csv(f"./model_stats/{modifier}_stats_table.csv", index=False)
    dated_estimates.to_csv(f"./model_stats/{modifier}_scatter_table.csv", index=False)

In [12]:
def drawCoefs(ax, coefs):
    full_wls = coefs.columns.astype(int)
    lmin, lmax = full_wls.min(), full_wls.max()
    coef_min = coefs.min()
    coef_max = coefs.max()*1.2
    xticks = list(range(lmin, lmax, 40)) + [790]
    ax.set_xlim(lmin, lmax)
    ax.set_xticks(xticks)
    ax.tick_params(axis='x', labelrotation=45)
    ax.set_ylabel(f"Coefficient", fontsize='x-large')
    ax.set_xlabel("Wavelength (nm)",  fontsize='x-large')
    ax.tick_params(labelsize='x-large')
    sns.lineplot(x=full_wls, y=coefs.iloc[0], ax=ax)

In [13]:
def makeLegend(legend):
    _size = 16
    handels = [lines.Line2D([], [], markerfacecolor='none', markeredgecolor='black', markersize=_size, marker='o', linestyle='None', label='Calibration'), 
                   lines.Line2D([], [], color='black', markersize=_size, marker='+', linestyle='None', label='Validation')]
    return handels+[lines.Line2D([], [], color=color, markersize=_size, marker='.', linestyle='None', label=name) for name, color in legend.items()]

def finalFig(drawFinals, stats, all_estimates, all_coefs, filename):
    rows = len(stats.Type)
    # top, bottom, left, right are coordinates
    spec = {"width_ratios":[1, 2], 'hspace': 0.25, 'bottom': 0.2, 'top': 0.98, 'left': 0.08, 'right': 0.95} 
    fig, axes = plt.subplots(rows, COLS, figsize=(SIZE*3, SIZE*rows), gridspec_kw = spec)
    
    
    nextNumber = 1
    ax = 0
    for predict in stats.Type:
        naxes = axes[ax]
        data = stats[stats.Type == predict]
        estimates = all_estimates[all_estimates.Type == predict]
        coefs = all_coefs[all_coefs.Type == predict]
        # same predict plots are one below the other instread of side by side
        # add and remove cols from next number to comply with this
        nextNumber = drawFinals(naxes, data, estimates, coefs, 
                                predict, num=nextNumber)
        ax += 1

    handles = makeLegend(COLORS)
    plt.figlegend(handles=handles, loc="lower center", ncol=len(handles), fontsize='large', bbox_to_anchor=(0.5, 0.1))
    txt = "(A)-(B) are for the chlorophyl a content, (C)-(D) are for chlorophyl b content and (E)-(F) are for total chlorophyl content.\n(A), (C), (E) are y-y plots for the PLS models. Statistics shown for model validation.\nLine of optimal fit is colored green, line of best fit for the validation data is colored red.\n(B), (D), (F) are coeffient plots of the wavelengths used in the model."
    fig.text(0.02, 0.02, txt, fontsize='large')
    
    plt.savefig(f'./graphs/{filename}.png')
    #plt.show()
    plt.close()

In [14]:
def finalGraphs(dates, colors):
    ''' supply inner function with location of flag leaf internal and external'''
    lx = 0.05
    ly = 0.93
    markers = {'cal': 'o', 'val': '+'}
    alpha=0.6
    units = r'($\mu gcm^{-2}$)'
    predict_text = {
        'Ca': 'Chl-a',
        'Cb': 'Chl-b',
        'TChl': 'TChl'
    }
    def inner(axes, data, estimates, coefs, predict, num):
        # valid and calib alpha & selected vars are the same. The plots expect the valid dataset

        est_cal = estimates[estimates.cal == 1]
        est_val = estimates[estimates.cal == 0]
        ax0 = axes[0]
        ax0.set_xlabel(f"Observed {predict_text[predict]} {units}", fontsize='large')
        ax0.set_ylabel(f"Predicted {predict_text[predict]} {units}", fontsize='large' )
        ax0.set_aspect(adjustable='box', aspect='equal')
        ax0.tick_params(labelsize='large')
        
        axLetter = chr(ord('`')+num)
        num += 1

        date_names = dates.unique()
        for date in date_names:
            date_indices = dates[dates == date].index
            date_cal = est_cal[est_cal.org_index.isin(date_indices)]
            date_val = est_val[est_val.org_index.isin(date_indices)]
            y_cal = date_cal.obs
            y_val = date_val.obs
            y_cal_hat = date_cal.pred
            y_val_hat = date_val.pred
            
            ax0.scatter(y_cal, y_cal_hat, facecolors='none', edgecolors=colors[date], marker=markers['cal'], alpha=alpha)
            ax0.scatter(y_val, y_val_hat, color=colors[date], marker=markers['val'], alpha=alpha)
        
        z_cal = np.polyfit(est_cal.obs, est_cal.pred, 1)
        z_val = np.polyfit(est_val.obs, est_val.pred, 1)
        
        ax0.plot(*get_line_ends(est_cal.obs, np.polyval(z_cal, est_cal.obs)), color='black', linestyle="solid", linewidth=1)
        ax0.plot(*get_line_ends(est_val.obs, np.polyval(z_val, est_val.obs)), color='black', linestyle=(0, (5, 10)), linewidth=1)
        
        ax0.plot((0, 1), (0, 1), linewidth=1, c='black', linestyle='dotted', transform=ax0.transAxes)
        ax0.set_yticks(ax0.get_xticks())
        ax0.set_ylim(ax0.get_xlim())
        
        R2C_str = r"─── $Cal: {R}^{2}$"
        R2V_str = r"─ ─ $Val: {R}^{2}$"
        R2C = data.R2C.iloc[0]
        R2V = data.R2V.iloc[0]
        ax0.text(0.53, 0.1, f"{R2C_str}={R2C:.2f}\n{R2V_str}={R2V:.2f}", fontsize='large', transform=ax0.transAxes)
        ax0.text(lx, ly, f"{axLetter}", weight='bold', fontsize='large', transform=ax0.transAxes)
        
        ax1 = axes[1]
        axLetter = chr(ord('`')+num)
        ax1.text(lx, ly, f"{axLetter}", weight='bold', fontsize='large', transform=ax1.transAxes)
        ax1.tick_params(labelsize='large')
        drawCoefs(ax1, coefs.drop(columns=['Type']))
        
        return num+1 # multiply cols by the number of plots
    return inner

In [18]:
drawFinals = finalGraphs(param_data['DAT'], COLORS)
finalFig(drawFinals, stats, estimates, coefs, f"{modifier}_graph_6_tiles")

## Ratio based on models

In [28]:
remove_514 = param_data[((param_data.Plots == 514) & (param_data.DAT == 81))].index[0]
switch_517 = param_data[((param_data.Plots == 517) & (param_data.DAT == 81))].index[0]
iestimates = estimates.set_index('org_index')
ca_sel = iestimates.Type == 'Ca'
cb_sel = iestimates.Type == 'Cb'
cal_sel = iestimates.cal == 1
ProtoCaCal = iestimates[ca_sel & cal_sel]
ProtoCaVal = iestimates[ca_sel & ~cal_sel]
CbCal = iestimates[cb_sel & cal_sel]
CbVal = iestimates[cb_sel & ~cal_sel]
# remove 514 from CaVal
# move 517 from CaCal to CaVal
CaVal = ProtoCaVal.drop(index=remove_514)
CaVal.loc[switch_517] = ProtoCaCal.loc[switch_517]
CaCal = ProtoCaCal.drop(index=switch_517)
ratio_data = param_data[['Ca', 'Cal_Val', 'Cb', 'DAT', 'Ratio', 'Treatment', 'Genotype']].copy()
ratio_data.loc[switch_517, 'Cal_Val'] = 'Val' # switch 517 to Val
cal_ratio_predict = CaCal.pred/CbCal.pred
val_ratio_predict = CaVal.pred/CbVal.pred
ratio_data.loc[ratio_data.Cal_Val == 'Cal', 'Pred_Ratio'] = cal_ratio_predict
ratio_data.loc[ratio_data.Cal_Val == 'Val', 'Pred_Ratio'] = val_ratio_predict
averaged_ratio = ratio_data.groupby(['DAT', 'Treatment', 'Genotype']).agg({'Ratio': 'mean', 'Pred_Ratio': 'mean'}).reset_index()
ratio_data['edgecolors'] = ratio_data.DAT.map(COLORS)
ratio_data['facecolors'] = ratio_data.apply(lambda row: row.edgecolors if row.Treatment == 'WW' else "none", axis=1)
averaged_ratio['edgecolors'] = averaged_ratio.DAT.map(COLORS)
averaged_ratio['facecolors'] = averaged_ratio.apply(lambda row: row.edgecolors if row.Treatment == 'WW' else "none", axis=1)

In [44]:
if SAVE_CSV:
    val_ratio_data[['Ratio', 'Pred_Ratio', 'Treatment', 'DAT']].to_csv(f"./model_stats/{modifier}_val_ratio.csv", index=False)
    averaged_ratio[['Ratio', 'Pred_Ratio', 'Treatment', 'DAT']].to_csv(f"./model_stats/{modifier}_avg_ratio.csv", index=False)

In [29]:
val_ratio_data = ratio_data[(ratio_data.Cal_Val == 'Val') & (~ratio_data.Pred_Ratio.isna())]
avg_ratio = averaged_ratio.Ratio
avg_pred_ratio = averaged_ratio.Pred_Ratio
avg_slope, avg_intercept, _, avg_p, _ = linregress(avg_ratio, avg_pred_ratio)
avg_r2 = 1-((((avg_ratio-avg_pred_ratio)**2).sum())/(((avg_ratio-avg_ratio.mean())**2).sum()))
avg_rmse = mse(avg_ratio, avg_pred_ratio, squared=False)
avg_minmax = np.array([avg_ratio.min(), avg_ratio.max()])

val_ratio = val_ratio_data.Ratio
val_pred_ratio = val_ratio_data.Pred_Ratio
val_slope, val_intercept, _, val_p, _ = linregress(val_ratio, val_pred_ratio)
val_r2 = 1-((((val_ratio-val_pred_ratio)**2).sum())/(((val_ratio-val_ratio.mean())**2).sum()))
val_rmse = mse(val_ratio, val_pred_ratio, squared=False)
val_minmax = np.array([val_ratio.min(), val_ratio.max()])

R2_str = r"${R}^{2}$"
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))
ax0 = axes[0]
ax = axes[1]
ax0.set_xlabel('Chl-a:b (observed)', fontsize='large')
ax0.set_ylabel('Chl-a:b (best PLSR)', fontsize='large')
ax0.set_aspect(adjustable='datalim', aspect='equal')
ax.set_xlabel('Chl-a:b (observed)', fontsize='large')
ax.set_ylabel('Chl-a:b (best PLSR)', fontsize='large')
ax.set_aspect(adjustable='datalim', aspect='equal')


ax0.scatter(x=val_ratio, y=val_pred_ratio, color=val_ratio_data.edgecolors, facecolor=val_ratio_data.facecolors)
ax0.plot([0, 1], [0, 1], color='black', linestyle='dotted', transform=ax0.transAxes) # 1:1 line
ax0.plot(val_minmax, val_minmax*val_slope+val_intercept, color='black') # regression line
ax0.text(s='a', x=0.02, y=0.95, weight='bold', fontsize='large', transform=ax0.transAxes)
ax0.text(s=f"n = {len(val_ratio_data)}\n{R2_str} = {val_r2:.2f}\nRMSE: {val_rmse:.2f}\np<0.0001", fontsize='large',\
         x=0.02, y=0.7, transform=ax0.transAxes)
ax0.set_yticks(ax0.get_xticks())
ax0.tick_params(labelsize='large')

ax.scatter(x=avg_ratio, y=avg_pred_ratio, color=averaged_ratio.edgecolors, facecolor=averaged_ratio.facecolors)
ax.plot([0, 1], [0, 1], color='black', linestyle='dotted', transform=ax.transAxes) # 1:1 line
ax.plot(avg_minmax, avg_minmax*avg_slope+avg_intercept, color='black') # regression line
ax.text(s='b', x=0.02, y=0.95, weight='bold', fontsize='large', transform=ax.transAxes)
ax.text(s=f"n = {len(averaged_ratio)}\n{R2_str} = {avg_r2:.2f}\nRMSE: {avg_rmse:.2f}\np<0.0001", fontsize='large',\
        x=0.02, y=0.7, transform=ax.transAxes)
ax.set_yticks(ax.get_xticks())
ax.tick_params(labelsize='large')

handles = [lines.Line2D([], [], markerfacecolor='none', markersize=16, markeredgecolor='black', marker='o', linestyle='None', label='WL'), 
                   lines.Line2D([], [], color='black', markersize=16, marker='o', linestyle='None', label='WW')]
handles += [lines.Line2D([], [], color=color, markersize=16, marker='.', linestyle='None', label=f'{name} DAT') for name, color in COLORS.items()]
plt.figlegend(handles=handles, loc="lower center", ncol=len(handles), bbox_to_anchor=(0.5, 0.15))
#fig.tight_layout()
fig.subplots_adjust(bottom=0.3)
fig.text(0.02, 0.02, '''1:1 graph of the relationship between observed Chl-a:b vs PLSR predicted Chl-a:b (i.e. Chl-a:b calculated from the best performing PLSR for Chl-a and Chl-b separately).
(a) presents result from calibration model on the 30 % validation dataset, and 
(b) presents averaged dataset from calibration model on all (100%) the dataset.''', fontsize='large', wrap=True)
plt.axis('square')
plt.savefig(f'./graphs/{modifier}_ratio.png')

In [306]:
plt.close()

In [326]:
bbox = ax0.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
width, height = bbox.width, bbox.height
(width-height)*2.54

0.10852727272727103